In [1]:
%matplotlib inline
%load_ext rpy2.ipython


import datetime as dt
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import statsmodels.api as sm
from pykalman import KalmanFilter
import numpy.ma as ma
import scipy.linalg as linalg

# pd.set_option('max_rows', 20)
# pd.set_option('max_rows', 1000)




#simul parameters
nm = 3
nq = 1
T = 20

In [2]:
%%R

library(MARSS)

# Building Matrices

Created this file because I forgot to commit the last file I\'ve used

Target: implement the MARSS Framework on Banbura Modugno 2010.


# MARSS

$$ x_t = B_t x_{t-1} + u_t + C_t c_t + w_t, \qquad where \quad w_t \sim MVN(0,Q_t) \qquad (1)$$
$$ y_t = Z_t x_t + a_t + D_t d_t + v_t, \qquad where v_t \sim MVN(0,R_t) \qquad (2)$$

* $x$ is a $m \times T$ matrix
* $w$ is $m \times T$ 
* $y$ is $n \times T$ 

Thus:

* m is the dimension of the state vector
* T is the sample length
* n is the total number of variables

# Banbura Modugno 2010

The first equation from Banbura Modugno corresponds to (2) and the second equation corresponds to (1)

![Banbura Modugno 2010](BanburaModugno.png)

# Dimensions

Lets build an example data set to work with

In [3]:
cols = ("m" + pd.DataFrame(list(range(0,nm))).astype(str))[0].tolist()
monthlyData = pd.DataFrame(np.random.rand(T,nm),columns=cols)
monthlyData

,m0,m1,m2
0,0.880156,0.114246,0.817552
1,0.406050,0.070118,0.184573
2,0.464014,0.541727,0.355364
3,0.739954,0.302707,0.632000
4,0.814830,0.872591,0.618599
5,0.146599,0.282675,0.424534
6,0.129219,0.821755,0.415943
7,0.929956,0.096302,0.520040
8,0.931574,0.131702,0.761709
9,0.033296,0.749558,0.936896


In [4]:
cols = ("q" + pd.DataFrame(list(range(0,nq))).astype(str))[0].tolist()
quarterlyData = pd.DataFrame(np.random.rand(T,nq),columns=cols)
quarterlyData

,q0
0,0.355193
1,0.221938
2,0.133211
3,0.773546
4,0.063292
5,0.330026
6,0.830694
7,0.554547
8,0.435729
9,0.981025


Mapping our dimensions:

In [5]:
T = monthlyData.shape[0]
T

20

In [6]:
n = (monthlyData.columns | quarterlyData.columns).shape[0]
n

4

In [7]:
nm = (monthlyData.columns).shape[0]
nm

3

In [8]:
nq = (quarterlyData.columns).shape[0]
nq

1

We will start with only one unobserved factor. Thus $f_t$ is $1 \times 1$.

In order to construct the following vector:

![State Vector](vec.png)

We need to remember that $\varepsilon_t^M$ is $n_M \times 1$ and $\varepsilon_t^Q$ is $n_Q \times 1$.

Also we have to include all 4 $\varepsilon_t^Q$'s lags

In [9]:
m = 5 + nm + 5 * nq
m

13

# Data Matrices

$y$ which is $n \times T$ will be the vertical stack of the monthly and quarterly variables

In [10]:
quarterlyData

,q0
0,0.355193
1,0.221938
2,0.133211
3,0.773546
4,0.063292
5,0.330026
6,0.830694
7,0.554547
8,0.435729
9,0.981025


In [11]:
monthlyData

,m0,m1,m2
0,0.880156,0.114246,0.817552
1,0.406050,0.070118,0.184573
2,0.464014,0.541727,0.355364
3,0.739954,0.302707,0.632000
4,0.814830,0.872591,0.618599
5,0.146599,0.282675,0.424534
6,0.129219,0.821755,0.415943
7,0.929956,0.096302,0.520040
8,0.931574,0.131702,0.761709
9,0.033296,0.749558,0.936896


In [12]:
y = pd.concat([ monthlyData.transpose(), quarterlyData.transpose(),], axis=0)
y

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
m0,0.880156,0.406050,0.464014,0.739954,0.814830,0.146599,0.129219,0.929956,0.931574,0.033296,0.972090,0.271814,0.303383,0.691934,0.741688,0.484056,0.736655,0.720056,0.737028,0.535695
m1,0.114246,0.070118,0.541727,0.302707,0.872591,0.282675,0.821755,0.096302,0.131702,0.749558,0.579336,0.318085,0.953248,0.581324,0.496923,0.884743,0.244827,0.075941,0.678627,0.880709
m2,0.817552,0.184573,0.355364,0.632000,0.618599,0.424534,0.415943,0.520040,0.761709,0.936896,0.109786,0.597630,0.271554,0.176549,0.683094,0.268026,0.933571,0.168833,0.373701,0.383514
q0,0.355193,0.221938,0.133211,0.773546,0.063292,0.330026,0.830694,0.554547,0.435729,0.981025,0.395369,0.524292,0.754473,0.955545,0.757247,0.017120,0.906260,0.348688,0.781784,0.371850


Verification:

In [13]:
(y.shape[0] == n) and (y.shape[1] == T)

True

# Coefficients

## Z Matrix

![Z Matrix](zMatrix.png)

The $Z$ matrix is a $n \times m$

The matrix we see on Banbura and Modugno is clearly n x m

Firt we have to generate a vector of m different coefficients named by strings. Which will be Banbura's $\Lambda_M$

In [14]:
coefs = []
for el in monthlyData.columns:
    coefs.append(el + "_loading")
lambdaM = pd.DataFrame(coefs)

To complete the first line of their matrix:

In [15]:
line1 = lambdaM
line1 = pd.concat([line1, pd.DataFrame(np.zeros((nm,4)))], axis=1)

line1 = pd.concat([line1, pd.DataFrame(np.identity(nm))], axis=1)
missingDimension = m - line1.shape[1]
line1 = pd.concat([line1, pd.DataFrame(np.zeros((nm,missingDimension)))], axis=1)

line1

,0,0,1,2,3,0,1,2,0,1,2,3,4
0,m0_loading,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,m1_loading,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,m2_loading,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


we are not yet sure how to implement linear restrictions on parameters. Fow now we are assuming that we should use a string like "2*a"

We will first create lambdaQ

For now this code suffices. But if we choose to use more quarterly series or more unobserved factors we would have to change it.

In [16]:
coefs = []
for el in quarterlyData.columns:
    coefs.append(el + "_loading")
lambdaQ = pd.DataFrame(coefs)

line2 = lambdaQ
line2 = pd.concat([line2, "2*" + lambdaQ], axis=1)
line2 = pd.concat([line2, "3*" + lambdaQ], axis=1)
line2 = pd.concat([line2, "2*" + lambdaQ], axis=1)
line2 = pd.concat([line2, lambdaQ], axis=1)

line2 = pd.concat([line2, pd.DataFrame(np.zeros((nq,nm)))], axis=1)

line2 = pd.concat([line2, pd.DataFrame(np.ones((nq,1)))], axis=1)
line2 = pd.concat([line2, 2*pd.DataFrame(np.ones((nq,1)))], axis=1)
line2 = pd.concat([line2, 3*pd.DataFrame(np.ones((nq,1)))], axis=1)
line2 = pd.concat([line2, 2*pd.DataFrame(np.ones((nq,1)))], axis=1)
line2 = pd.concat([line2, pd.DataFrame(np.ones((nq,1)))], axis=1)

line2

,0,0,0,0,0,0,1,2,0,0,0,0,0
0,q0_loading,2*q0_loading,3*q0_loading,2*q0_loading,q0_loading,0.0,0.0,0.0,1.0,2.0,3.0,2.0,1.0


In [17]:
line1.columns = list(range(0,line1.columns.shape[0]))
line2.columns = list(range(0,line2.columns.shape[0]))

Z = pd.concat([line1,line2], axis=0, ignore_index=True)
Z = Z.apply(pd.to_numeric,1,errors='ignore')
Z = Z.apply(pd.to_numeric,0,errors='ignore')
Z

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,m0_loading,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,m1_loading,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,m2_loading,0,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,q0_loading,2*q0_loading,3*q0_loading,2*q0_loading,q0_loading,0.0,0.0,0.0,1.0,2.0,3.0,2.0,1.0


## B Matrix

![B matrix](bMatrix.png)

$B$ matrix is $m \times m$

Since we're using only one unobserved factor, for the time being $A_t$ is a scalar and $r=1$.

In [18]:
bMat = np.array(["A1",0.,0.,0.])
bMat = np.vstack((np.array(bMat),np.identity(4)))
bMat = linalg.block_diag(bMat,np.array([0]))

alphaM = "alphaM_" + pd.DataFrame([range(0,nm)]).astype(str) 
alphaM = alphaM.transpose()[0]
alphaM = np.diag(alphaM)

bMat = bMat[:-1]

bMat = linalg.block_diag(bMat,alphaM)

alphaQ = "alphaQ_" + pd.DataFrame([range(0,nq)]).astype(str) 
alphaQ = alphaQ.transpose()[0]
alphaQ = np.diag(alphaQ)

bMat = linalg.block_diag(bMat,alphaQ)

newLine = np.hstack((np.zeros((nq,bMat.shape[1]-nq)),np.identity(nq)))
bMat = np.vstack((bMat,newLine))

bMat = linalg.block_diag(bMat,np.identity(m-bMat.shape[1]-1))
bMat = linalg.block_diag(bMat,np.array([0]))
bMat = bMat[:-1]
B = pd.DataFrame(bMat)
B = B.apply(pd.to_numeric,1,errors='ignore')
B = B.apply(pd.to_numeric,0,errors='ignore')
# alphaM
# newLine
# pd.to_numeric(pd.DataFrame(bMat)[0].iloc[1])
B

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,A1,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0
2,0,1.0,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0
3,0,0.0,1.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0
4,0,0.0,0.0,1.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0
5,0,0.0,0.0,0.0,0.0,alphaM_0,0,0,0,0.0,0.0,0.0,0.0
6,0,0.0,0.0,0.0,0.0,0,alphaM_1,0,0,0.0,0.0,0.0,0.0
7,0,0.0,0.0,0.0,0.0,0,0,alphaM_2,0,0.0,0.0,0.0,0.0
8,0,0.0,0.0,0.0,0.0,0,0,0,alphaQ_0,0.0,0.0,0.0,0.0
9,0,0.0,0.0,0.0,0.0,0,0,0,1,0.0,0.0,0.0,0.0


Verification:

In [19]:
(B.shape[0] == m) and (B.shape[1] == m)

True

## Q Matrix

$Q_t$ is $m \times =m$

![Error Vector](error.png)

It has to assure that the zeros are zeros. and the others might or might not covariate, we should test it.

In [20]:
u = np.array("u_t")
u = np.vstack((u,np.zeros((4,1))))
merrors = "em_" + pd.DataFrame(np.array([list(range(0,nm))])).astype(str)
u = np.vstack((u,merrors.T))

merrors = "eq_" + pd.DataFrame(np.array([list(range(0,nq))])).astype(str)
u = np.vstack((u,merrors.T))

u = np.vstack((u,np.zeros((m-u.shape[0],1))))

u = pd.DataFrame(u)

u[0] = pd.to_numeric(u[0],errors='ignore')

u

,0
0,u_t
1,0.0
2,0.0
3,0.0
4,0.0
5,em_0
6,em_1
7,em_2
8,eq_0
9,0


In [21]:
idx = np.unique(np.where(u=="0.0" )[0].tolist() + np.where(u==0 )[0].tolist())
idx
# pd.to_numeric(u,errors='ignore')
# u.iloc[9]

array([ 1,  2,  3,  4,  9, 10, 11, 12])

In [22]:
Q = "Q_" + pd.DataFrame([range(0,m)]).astype(str) 
Q = Q.transpose()[0]
Q = np.diag(Q)
Q[idx] = 0
Q = pd.DataFrame(Q)
Q

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,Q_0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,Q_5,0,0,0,0,0,0,0
6,0,0,0,0,0,0,Q_6,0,0,0,0,0,0
7,0,0,0,0,0,0,0,Q_7,0,0,0,0,0
8,0,0,0,0,0,0,0,0,Q_8,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0


Verification:

In [23]:
(u.shape[0] == m) and (u.shape[1] == 1)

True

# Simulation Trial

In [24]:
Zi = Z.as_matrix().T.flatten()
Bi = B.as_matrix().T.flatten()
Qi = Q.as_matrix().T.flatten()
ui = u.as_matrix()
yi = y.as_matrix()


In [25]:
B.shape

(13, 13)

In [26]:
Z.shape

(4, 13)

In [27]:
y.shape

(4, 20)

In [29]:
%%R -i Zi,Bi,ui,yi,Z,Qi

# class(meas)

# Z="unconstrained"
# B="diagonal and unequal"
# B="identity"
# B="unconstrained"
# x0=matrix(c("pi1"),1,1)
# x0=matrix(c("pi1","pi2"),2,1)

# Z2=matrix(list("z1","z2","z3","z4","z5","z6","z7","z8","z9","z10","z11","z12","z13"),13,1)
Zi=matrix(Zi,4,13)
Bi=matrix(Bi,13,13)
Qi=matrix(Qi,13,13)
# print(Zi)
# print(Bi)


model.gen=list(Z=Zi,B=Bi,Q=Qi,A="zero",x0="zero",U="zero")
# model.gen=list(Z=Z,B=B,A="unconstrained",x0="zero",U="unconstrained")
# model.gen=list(Z=Z,B=B,A="zero",x0="zero",U="zero")
# model.gen=list(Z="diagonal",B="diagonal",A="zero",x0="zero",U="zero")
# model.gen=list(Z=Z)
# kemfit = MARSS(yi, model=model.gen,)
# kemfit = MARSS(y,)
kemfit = MARSS(yi, model=model.gen,control=list(maxit=1500,conv.test.slope.tol=0.00001,abstol=0.00001))
# kemfit = MARSS(y, model=model.gen,control=list(conv.test.slope.tol=0.00001,abstol=0.00001))
# states = kemfit$states
states = print(kemfit, what="model")
# message(kemfit, what="model")
# print(kemfit, what="start")
# print(kemfit, what="states")
# print(kemfit, what="ytT")
# print(kemfit, what="states.se")
# print(kemfit, what="kfs")
# print(kemfit$par$Z)
# message("porra")
# states=""
# cat("teste")

In [30]:
%%R

summary(Z)

PS: Nao tenho certeza se a matriz Q é diagonal